In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [3]:
data = data.drop(columns = ['RowNumber','CustomerId','Surname','Exited'],axis=1)
data.head()

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42       2       0.00              1   
1          608     Spain  Female   41       1   83807.86              1   
2          502    France  Female   42       8  159660.80              3   
3          699    France  Female   39       1       0.00              2   
4          850     Spain  Female   43       2  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1        101348.88  
1          0               1        112542.58  
2          1               0        113931.57  
3          0               0         93826.63  
4          1               1         79084.10

In [4]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [6]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df], axis=1)
data

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619       0   42       2       0.00              1          1   
1             608       0   41       1   83807.86              1          0   
2             502       0   42       8  159660.80              3          1   
3             699       0   39       1       0.00              2          0   
4             850       0   43       2  125510.82              1          1   
...           ...     ...  ...     ...        ...            ...        ...   
9995          771       1   39       5       0.00              2          1   
9996          516       1   35      10   57369.61              1          1   
9997          709       0   36       7       0.00              1          0   
9998          772       1   42       3   75075.31              2          1   
9999          792       0   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0                  1        101348.88               1.0                0.0   
1                  1        112542.58               0.0                0.0   
2                  0        113931.57               1.0                0.0   
3                  0         93826.63               1.0                0.0   
4                  1         79084.10               0.0                0.0   
...              ...              ...               ...                ...   
9995               0         96270.64               1.0                0.0   
9996               1        101699.77               1.0                0.0   
9997               1         42085.58               1.0                0.0   
9998               0         92888.52               0.0                1.0   
9999               0         38190.78               1.0                0.0   

      Geography_Spain  
0                 0.0  
1                 1.0  
2                 0.0  
3                 0.0  
4                 1.0  
...               ...  
9995              0.0  
9996              0.0  
9997              0.0  
9998              0.0  
9999              0.0  

[10000 rows x 12 columns]

In [7]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
## ANN with Regression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [11]:
# build the model
# after hyperparameter tuning the ANN model
model = Sequential([
    Dense(16,activation='relu', input_shape = (X_train.shape[1],)), # HL1
    Dense(16, activation='relu'), # HL2
    Dense(1) # output layer for regression (bydefalut activation="linear_activation")
])

# compile the model
model.compile(optimizer= 'adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

d:\Project-2-Bank-Churn-Modeling\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 481 (1.88 KB)

 Trainable params: 481 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# from tensorflow.keras.callbacks import EarlyStopping
# import datetime

# early_stopping_callback = EarlyStopping(monitor='val_loss',patience = 10, restore_best_weights = True)

In [12]:
# train the model
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs = 50)

Epoch 1/50


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100155.7969 - mae: 100155.7969 - val_loss: 98700.7266 - val_mae: 98700.7266
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100506.5547 - mae: 100506.5547 - val_loss: 98510.7031 - val_mae: 98510.7031
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99376.3438 - mae: 99376.3438 - val_loss: 97967.4844 - val_mae: 97967.4844
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99261.7812 - mae: 99261.7812 - val_loss: 96950.7812 - val_mae: 96950.7812
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98126.9297 - mae: 98126.9297 - val_loss: 95394.0625 - val_mae: 95394.0625
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 96964.0859 - mae: 96964.0859 - val_loss: 93277.6641 - val_mae: 93277.6641
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 93782.1172 - mae: 93782.1172 - val_loss: 90603.0234 - val_mae: 90603.0234
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 91705.296

In [13]:
## evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50928.8164 - mae: 50928.8164
Test MAE: 50486.84375


In [14]:
model.save('regressionModel.h5')